In [1]:
import numpy as np
import toml
from typing import List
from jinja2 import Template
import os
import moleculegraph

In [2]:
class LAMMPS_input():

    def __init__(self, mol_list: List, ff_path: str, playmol_ff_path: str, playmol_template: str):
        """
        Initilizing LAMMPS input class. Create playmol ff and save system independent force field parameters

        Args:
            mol_list (List): List containing moleculegraph objects for the component(s).
            ff_path (str): Path to toml file containing used force-field readable format by moleculegraph.
            playmol_ff_path (str): Path were the new playmol force field file should be writen to.
            playmol_template (str): Path to playmol template for system building.
        """

        # Save moleclue graphs of both components class wide
        self.mol_list = mol_list

        # Read in force field toml file
        with open(ff_path) as ff_toml_file:
            self.ff = toml.load(ff_toml_file)

        ## Specify force field parameters for all interactions seperately (nonbonded, bonds, angles and torsions)

        # Get (unique) atom types and parameters #
        self.nonbonded = np.array([j for sub in [molecule.map_molecule( molecule.unique_atom_keys, self.ff["atoms"] ) for molecule in mol_list] for j in sub])
        # Get (unique) bond types and parameters #
        self.bonds     = [j for sub in [molecule.map_molecule( molecule.unique_bond_keys, self.ff["bonds"] ) for molecule in mol_list] for j in sub]
        # Get (unique) angle types and parameters #     
        self.angles    = [j for sub in [molecule.map_molecule( molecule.unique_angle_keys, self.ff["angles"] ) for molecule in mol_list] for j in sub]
        # Get (unique) torsion types and parameters #
        self.torsions  = [j for sub in [molecule.map_molecule( molecule.unique_torsion_keys, self.ff["torsions"] ) for molecule in mol_list] for j in sub]
        
        if not all( [ all([bool(entry) for entry in self.nonbonded]), all([bool(entry) for entry in self.bonds]), all([bool(entry) for entry in self.angles]), all([bool(entry) for entry in self.torsions]) ] ):
            txt = "nonbonded" if not all([bool(entry) for entry in self.nonbonded]) else "bonds" if not all([bool(entry) for entry in self.bonds]) else "angles" if not all([bool(entry) for entry in self.angles]) else "torsions"
            raise ValueError("Something went wrong during the force field typing for key: %s"%txt)
        
        # Get all atom types not only the unique one
        self.ff_all    = np.array([j for sub in [molecule.map_molecule( molecule.atom_names, self.ff["atoms"] ) for molecule in mol_list] for j in sub])


        ## Define general settings that are not system size dependent ##

        self.renderdict              = {}
        molecule1                    = mol_list[0]

        # Definitions for atoms in the system 
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_atoms                    = [ 1, 1 + len(molecule1.unique_atom_indexes) ]

        self.number_of_atoms         = [ mol.atom_number for mol in mol_list ]
        self.atoms_running_number    = np.concatenate( [mol.unique_atom_inverse + add_atoms[i] for i,mol in enumerate(mol_list)], axis=0 )
        
        # Definitions for bonds in the system #
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_bonds = [ 1, 1+ len(molecule1.unique_bond_keys) ]

        self.number_of_bonds         = [ len(mol.bond_keys) for mol in mol_list ]
        self.bonds_running_number    = np.concatenate( [mol.unique_bond_inverse + add_bonds[i] for i,mol in enumerate(mol_list)], axis=0 )
        self.bond_numbers            = np.concatenate( [mol.bond_list + 1 for mol in mol_list], axis=0 )
        self.bond_numbers_ges        = np.unique( self.bonds_running_number )

        self.renderdict["bond_styles"]        = list( np.unique( [ p["style"] for p in self.bonds ] ) )
        self.renderdict["bond_type_number"]   = len( self.bond_numbers_ges )

        # Definitions for angles in the system 
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_angles = [ 1, 1+ len(molecule1.unique_angle_keys) ]

        self.number_of_angles        =[ len(mol.angle_keys) for mol in mol_list ]
        self.angles_running_number   = np.concatenate( [mol.unique_angle_inverse + add_angles[i] for i,mol in enumerate(mol_list)], axis=0 )
        self.angle_numbers           = np.concatenate( [mol.angle_list + 1 for mol in mol_list], axis=0 )
        self.angle_numbers_ges       = np.unique( self.angles_running_number )

        self.renderdict["angle_styles"]        = list( np.unique( [ p["style"] for p in self.angles] ) )
        self.renderdict["angle_type_number"]   = len(self.angle_numbers_ges)

        # Definitions for torsions in the system 
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_torsions = [ 1, 1+ len(molecule1.unique_torsion_keys) ]
        
        self.number_of_torsions      = [ len(mol.torsion_keys) for mol in mol_list ]
        self.torsions_running_number = np.concatenate( [mol.unique_torsion_inverse + add_torsions[i] for i,mol in enumerate(mol_list)], axis=0 )
        self.torsion_numbers         = np.concatenate( [mol.torsion_list + 1 for mol in mol_list], axis=0 )
        self.torsion_numbers_ges     = np.unique( self.torsions_running_number )

        self.renderdict["torsion_styles"]      = list( np.unique( [ p["style"] for p in self.torsions ] ) )
        self.renderdict["torsion_type_number"] = len(self.torsion_numbers_ges)


        ## Prepare dictionary for jinja2 template to write force field input for Playmol ##

        renderdict              = {}
        renderdict["nonbonded"] = list(zip([j for sub in [molecule.unique_atom_keys for molecule in mol_list] for j in sub],self.nonbonded))
        renderdict["bonds"]     = list(zip([j for sub in [molecule.unique_bond_names for molecule in mol_list] for j in sub],self.bonds))
        renderdict["angles"]    = list(zip([j for sub in [molecule.unique_angle_names for molecule in mol_list] for j in sub],self.angles))
        renderdict["torsions"]  = list(zip([j for sub in [molecule.unique_torsion_names for molecule in mol_list] for j in sub],self.torsions))
        
        # Generate force field file for playmol using jinja2 template
        os.makedirs( os.path.dirname(playmol_ff_path), exist_ok=True )

        with open(playmol_template) as file_:
            template = Template(file_.read())
        
        rendered = template.render( rd=renderdict )

        with open(playmol_ff_path, "w") as fh:
            fh.write(rendered) 
        
        print(renderdict)
        return


    def data_file_init(self, nmol_list: List, densitiy: float, decoupling: bool=False):
        """
        Function that prepares the LAMMPS data file at a given density for a given force field and system

        Args:
            nmol_list (list): List containing the number of molecules per component
            densitiy (float): Mass density of the component/mixture at that state [kg/m^3]
            decoupling (bool,optional): If a decoupling input should be prepared. This will introduce new types for 
                                        the last molecule of component 1, to ensure that pair wise interactions can be 
                                        coupled / decoupled using a lambda parameter. Defaults to False.
                              
        """

        # Variables defined here are used class wide 
        self.nmol_list      = nmol_list
        self.density        = densitiy
        
        molecule1,molecule2 = self.mol_list

        # Zip objects has to be refreshed for every system since its only possible to loop over them once
        self.renderdict["bond_paras"]    = zip(self.bond_numbers_ges, self.bonds)
        self.renderdict["angle_paras"]   = zip(self.angle_numbers_ges, self.angles)
        self.renderdict["torsion_paras"] = zip(self.torsion_numbers_ges, self.torsions)
        

        #### System specific settings ####

        ## Definitions for atoms in the system ##

        # In case coupling simulations are performed:
        # Differentiate the last molecule of component 1 from every other 
        # Therefore create new types for last molecule and increase the type number of every atom in species 2 aswell

        if ( nmol_list[0] >1 and decoupling ):
            self.add_type     = np.max( self.atoms_running_number[molecule1.atom_numbers] )
            a_list1           = np.arange( len(molecule1.unique_atom_keys) ) + 1
            a_list1_add       = np.arange( len(molecule1.unique_atom_keys) ) + 1 + self.add_type
            a_list2           = np.arange( len(molecule2.unique_atom_keys) ) + 1 + self.add_type + len(molecule1.unique_atom_keys)

            self.pair_numbers = list(a_list1) + list(a_list1_add) + list(a_list2)
            self.pair_ff      = list(self.nonbonded[a_list1-1])*2  + list(self.nonbonded[a_list2-1-self.add_type])

        else:
            self.add_type     = 0
            a_list1           = np.arange( len(molecule1.unique_atom_keys) ) + 1
            a_list2           = np.arange( len(molecule2.unique_atom_keys) ) + 1 + self.add_type + len(molecule1.unique_atom_keys)

            self.pair_numbers = list(self.a_list1) + list(self.a_list2)
            self.pair_ff      = list(self.nonbonded)

        self.renderdict["atom_paras"]       = list( zip(self.pair_numbers,self.pair_ff) )
        self.renderdict["atom_type_number"] = len(self.nonbonded) + self.add_type

        # Total atoms in system 
        self.total_number_of_atoms    = np.dot( self.number_of_atoms, nmol_list )

        # Total bonds in system 
        self.total_number_of_bonds    = np.dot( self.number_of_bonds, nmol_list )
        
        # Total angles in system 
        self.total_number_of_angles   = np.dot( self.number_of_angles, nmol_list) 
        
        # Total torsions in system 
        self.total_number_of_torsions = np.dot( self.number_of_torsions, nmol_list )

        ## Mass, mol, volume and box size of the system ##

        # Molar masses of each species [g/mol]
        #if not a["name"] == "H"
        Mol_masses = np.array( [ np.sum( [ a["mass"] for a in molecule.map_molecule( molecule.atom_names, self.ff["atoms"] ) ] ) for molecule in self.mol_list ] )

        # Account for mixture density --> in case of pure component this will not alter anything

        # mole fraction of mixture (== numberfraction)
        x = np.array( nmol_list ) / np.sum( nmol_list )

        # Average molar weight of mixture [g/mol]
        M_avg = np.dot( x, Mol_masses )

        # Total mole n = N/NA [mol] #
        n = np.sum( nmol_list ) / Avogadro

        # Total mass m = n*M [kg]
        mass = n * M_avg / 1000

        # Compute box volume V=m/rho and with it the box lenght L (in Angstrom) --> assuming orthogonal box
        # With mass (kg) and rho (kg/m^3 --> convert in g/A^3 necessary as lammps input)

        # Volume = mass / mass_density = mol / mol_density [A^3]
        volume = mass / self.density * 1e30

        boxlen = volume**(1/3) / 2

        box = [ -boxlen, boxlen ]

        self.renderdict["box_x"] = box
        self.renderdict["box_y"] = box
        self.renderdict["box_z"] = box
        
        return
    
def write_lammps_data(self,xyz_path,data_path):
        """
        Function that generates LAMMPS data file, containing bond, angle and torsion parameters, as well as all the coordinates etc.

        Args:
            xyz_path (str): Path where xyz file is located for this system.
            data_path (str): Path where the LAMMPS data file should be generated.
        """

        molecule1,molecule2 = self.mol_list

        atom_count       = 0
        bond_count       = 0
        angle_count      = 0
        torsion_count    = 0
        mol_count        = 0
        mol_count1       = 0
        coordinate_count = 0
        
        lmp_atom_list    = []
        lmp_bond_list    = []
        lmp_angle_list   = []
        lmp_torsion_list = []

        coordinates = moleculegraph.funcs.read_xyz(xyz_path)

        for m,mol in enumerate(self.mol_list):

            # All force field lists are created for both species --> only take part of the list for the specific component 

            # First component
            if m == 0:
                idx  = mol.atom_numbers
                idx1 = np.arange( len(self.mol_list[0].bond_keys) )
                idx2 = np.arange( len(self.mol_list[0].angle_keys) )
                idx3 = np.arange( len(self.mol_list[0].torsion_keys) )

            # Second component (add to the indices of the second component the number of atoms, bonds, angles and torsions of first component)
            elif m == 1:
                idx   = mol.atom_numbers + self.mol_list[0].atom_number
                idx1  = np.arange( len(self.mol_list[1].bond_keys) ) + len(self.mol_list[0].bond_keys)
                idx2  = np.arange( len(self.mol_list[1].angle_keys) ) + len(self.mol_list[0].angle_keys)
                idx3  = np.arange( len(self.mol_list[1].torsion_keys) ) + len(self.mol_list[0].torsion_keys)


            ## Now write LAMMPS input for every molecule of each component ##

            for mn in range(self.nmol_list[m]):
                
                # Define atoms

                for atomtype,ff_atom in zip(self.atoms_running_number[idx],self.ff_all[idx]):
                    
                    atom_count +=1
                    
                    # These are coupling specific settings. If no coupling is wanted, add_type is 0 and no changes occur
                    # If last molecule of species 1: Add to every atomtype the number of types of species 1
                    if mn == self.nmol_list[0]-1 and m == 0:
                        atomtype += self.add_type
                    # If atoms of species 2, add to there atomtype to account for the extra types introduced (if N(species)==1 no special types are added (add_type =0))
                    elif m==1:
                        atomtype += self.add_type

                    # LAMMPS INPUT: total n° of atom in system, mol n° in System, atomtype, partial charges,coordinates
                    
                    line = [ atom_count, mol_count+1, atomtype, ff_atom["charge"],*coordinates[coordinate_count]["xyz"],
                            coordinates[coordinate_count]["atom"] ]

                    lmp_atom_list.append(line)

                    coordinate_count += 1

                # Define bonds

                for bondtype,bond in zip(self.bonds_running_number[idx1],self.bond_numbers[idx1]):

                    bond_count += 1

                    # The bond counting for 2nd species needs to start after nmol1 * bonds_mol1 for right index of atoms

                    if m == 0:
                        dummy = bond + mol_count * mol.atom_number
                        txt   = [ "A"+str(x) for x in bond ]
                    elif m == 1:
                        dummy = bond + (mol_count-mol_count1) * mol.atom_number + mol_count1 * self.mol_list[0].atom_number
                        txt   = [ "A"+str(x+molecule1.atom_number) for x in bond ]

                    # LAMMPS INPUT: total n° of bonds in system, bond type, atoms of system with this bond type

                    line = [ bond_count, bondtype, *dummy, *txt]

                    lmp_bond_list.append(line)

                # Define angles

                for angletype,angle in zip(self.angles_running_number[idx2],self.angle_numbers[idx2]):

                    angle_count += 1

                    # The angles counting for 2nd species needs to start after nmol1 * angles_mol1 for right index of atoms

                    if m == 0:
                        dummy = angle + mol_count * mol.atom_number
                        txt   = ["A"+str(x) for x in angle]
                    elif m == 1:
                        dummy = angle + (mol_count-mol_count1) * mol.atom_number + mol_count1 * self.mol_list[0].atom_number
                        txt   = ["A"+str(x+molecule1.atom_number) for x in angle]

                    # LAMMPS INPUT: total n° of angles in system, angle type, atoms of system with this angle type

                    line = [ angle_count, angletype, *dummy, *txt]

                    lmp_angle_list.append(line)

                # Define torsions

                for torsiontype,torsion in zip(self.torsions_running_number[idx3],self.torsion_numbers[idx3]):

                    torsion_count += 1

                    # The torsion counting for 2nd species needs to start after nmol1 * torsion_mol1 for right index of atoms

                    if m == 0:
                        dummy = torsion + mol_count * mol.atom_number
                        txt   = ["A"+str(x) for x in torsion]
                    elif m == 1:
                        dummy = torsion + (mol_count-mol_count1) * mol.atom_number + mol_count1 * self.mol_list[0].atom_number
                        txt   = ["A"+str(x+molecule1.atom_number) for x in torsion]

                    dummy = np.flip( dummy ) # (?)
                    torsion = np.flip( torsion)  # (?)

                    # LAMMPS INPUT: total n° of torsions in system, torsion type, atoms of system with this torsion type

                    line = [ torsion_count, torsiontype, *dummy, *txt]

                    lmp_torsion_list.append(line)


                if m == 0: mol_count1 += 1

                mol_count += 1

                
            self.renderdict["atoms"]    = lmp_atom_list
            self.renderdict["bonds"]    = lmp_bond_list
            self.renderdict["angles"]   = lmp_angle_list
            self.renderdict["torsions"] = lmp_torsion_list

            self.renderdict["atom_number"]    = atom_count
            self.renderdict["bond_number"]    = bond_count
            self.renderdict["angle_number"]   = angle_count
            self.renderdict["torsion_number"] = torsion_count
            
        # Write to jinja2 template file to create LAMMPS input file

        if not os.path.exists(data_path[:data_path.rfind("/")+1]): os.makedirs(data_path[:data_path.rfind("/")+1])
        
        with open("../templates/template.lmp") as file_:
            template = Template(file_.read())
            
        rendered = template.render( rd = self.renderdict )

        with open(data_path, "w") as fh:
            fh.write(rendered)

        return

In [ ]:
## TO do: debug the case where no bonds, angles and torsions are there and i use np.concatenate
## To do: add a info, that at bonds its not # A1 A2 rather # cH_alc OH_alc

In [33]:
ff = "./force-fields/forcefield_lammps.toml"

m1 = "[cH_alcohol][OH_alcohol][CH2_alcohol][CH2_alcohol][OH_alcohol][cH_alcohol]"
m2 = "[cH_alcohol][OH_alcohol][CH2_alcohol][CH2_alcohol]"
m1 = moleculegraph.molecule(m1)
m2 = moleculegraph.molecule(m2)
m_list = [m1,m2]
a = LAMMPS_input( m_list, ff, "." )

{'nonbonded': [('cH_alcohol', {'name': 'H', 'mass': 1.0, 'epsilon': 0.0, 'sigma': 1.0, 'm': 12.0, 'cut': 0.0, 'charge': 0.404, 'style': 'hybrid Mie/cut coul/long'}), ('OH_alcohol', {'name': 'OH_alcohol', 'mass': 17.007, 'epsilon': 0.1673, 'sigma': 3.035, 'm': 12.0, 'cut': 14.0, 'charge': -0.65, 'style': 'hybrid Mie/cut coul/long'}), ('CH2_alcohol', {'name': 'CH2_alcohol', 'mass': 14.027, 'epsilon': 0.1673, 'sigma': 3.842, 'm': 14.0, 'cut': 14.0, 'charge': 0.246, 'style': 'hybrid Mie/cut coul/long'}), ('cH_alcohol', {'name': 'H', 'mass': 1.0, 'epsilon': 0.0, 'sigma': 1.0, 'm': 12.0, 'cut': 0.0, 'charge': 0.404, 'style': 'hybrid Mie/cut coul/long'}), ('OH_alcohol', {'name': 'OH_alcohol', 'mass': 17.007, 'epsilon': 0.1673, 'sigma': 3.035, 'm': 12.0, 'cut': 14.0, 'charge': -0.65, 'style': 'hybrid Mie/cut coul/long'}), ('CH2_alcohol', {'name': 'CH2_alcohol', 'mass': 14.027, 'epsilon': 0.1673, 'sigma': 3.842, 'm': 14.0, 'cut': 14.0, 'charge': 0.246, 'style': 'hybrid Mie/cut coul/long'})], 'b